In [3]:
import os
import sys

sys.path.append( os.path.abspath("../../panda") )

from time import time, sleep                                                                      
import struct                                                                                     
import argparse                                                                                   
import contextlib                                                                                 
import io                                                                                         
                                                                                                  
from prompt_toolkit import PromptSession                                                          
from prompt_toolkit.history import FileHistory                                                    
from panda import Panda  

def init(bus, speed_kbps, silent=False):                                                          
   f = io.StringIO()                                                                               
   with contextlib.redirect_stdout(f):                                                             
     panda = Panda()                                                                               
   if DEBUG or not silent: print(f.getvalue())                                                     
   if bus == -1:                                                                                   
     bus = 1 if panda.has_obd() else 0                                                             
   if DEBUG: print(f'--- BUS: {bus}')                                                              
   if speed_kbps:                                                                                  
     if DEBUG: print(f'--- SPEED: {speed_kbps} kbps')                                              
     panda.set_can_speed_kbps(bus, speed_kbps)                                                     
   panda.set_safety_mode(Panda.SAFETY_ALLOUTPUT)                                                   
   return panda, bus    

DEBUG = 1
CAN_BUS = 0
SPEED_KBPS = 500

bus = CAN_BUS
is_tty = sys.stdin.isatty() 
panda, bus = init(CAN_BUS, SPEED_KBPS, silent=not is_tty)

panda.can_clear(bus)    #flash TX
panda.can_clear(0xFFFF) #flash RX
start_time = time()
i = 0
while time() - start_time < 1:
    msgs = panda.can_recv()
    # can_id , ts(us), data, bus 
    for rx_addr, ts, dat, src in msgs:
        i += 1
        if DEBUG: print(f'\n---> RX {src} |{ts}| {hex(rx_addr)}: 0x{dat.hex()} {i}')
                        
print(panda,bus)

panda.close()

opening device 410020000651363038363036 0xddcc
connected

--- BUS: 0
--- SPEED: 500 kbps

---> RX 0 |5917| 0x102: 0x27 1

---> RX 130 |4452| 0x102: 0x27 2

---> RX 2 |31079| 0x120: 0xf7 3

---> RX 128 |32662| 0x120: 0xf7 4

---> RX 0 |55957| 0x102: 0x28 5

---> RX 130 |54492| 0x102: 0x28 6

---> RX 2 |15588| 0x120: 0xf8 7

---> RX 128 |17171| 0x120: 0xf8 8

---> RX 0 |40488| 0x102: 0x29 9

---> RX 130 |39023| 0x102: 0x29 10

---> RX 2 |95| 0x120: 0xf9 11

---> RX 128 |1681| 0x120: 0xf9 12

---> RX 0 |24996| 0x102: 0x2a 13

---> RX 130 |23529| 0x102: 0x2a 14

---> RX 2 |50143| 0x120: 0xfa 15

---> RX 128 |51727| 0x120: 0xfa 16

---> RX 0 |9521| 0x102: 0x2b 17

---> RX 130 |8056| 0x102: 0x2b 18

---> RX 2 |34656| 0x120: 0xfb 19

---> RX 128 |36242| 0x120: 0xfb 20

---> RX 0 |59563| 0x102: 0x2c 21

---> RX 130 |58096| 0x102: 0x2c 22

---> RX 2 |19157| 0x120: 0xfc 23

---> RX 128 |20743| 0x120: 0xfc 24

---> RX 0 |44075| 0x102: 0x2d 25

---> RX 130 |42609| 0x102: 0x2d 26

---> RX 2 |3690| 

In [28]:
#drivers/can.h
# // assign CAN numbering
# // bus num: Can bus number on ODB connector. Sent to/from USB
# //    Min: 0; Max: 127; Bit 7 marks message as receipt (bus 129 is receipt for but 1)
# // cans: Look up MCU can interface from bus number
# // can number: numeric lookup for MCU CAN interfaces (0 = CAN1, 1 = CAN2, etc);
# // bus_lookup: Translates from 'can number' to 'bus number'.
# // can_num_lookup: Translates from 'bus number' to 'can number'.
# // can_forwarding: Given a bus num, lookup bus num to forward to. -1 means no forward.

# // Panda:       Bus 0=CAN1   Bus 1=CAN2   Bus 2=CAN3
# CAN_TypeDef *cans[] = {CAN1, CAN2, CAN3};
# uint8_t bus_lookup[] = {0,1,2};
# uint8_t can_num_lookup[] = {0,1,2,-1};
# int8_t can_forwarding[] = {-1,-1,-1,-1};
# uint32_t can_speed[] = {5000, 5000, 5000, 333};
# #define CAN_MAX 3U

# #define CANIF_FROM_CAN_NUM(num) (cans[num])
# #define CAN_NUM_FROM_CANIF(CAN) ((CAN)==CAN1 ? 0 : ((CAN) == CAN2 ? 1 : 2))
# #define BUS_NUM_FROM_CAN_NUM(num) (bus_lookup[num])
# #define CAN_NUM_FROM_BUS_NUM(num) (can_num_lookup[num])

#@can_rx()
#can_forwarding[] = {-1,-1,-1,-1};
#void can_set_forwarding(int from, int to) {
#   can_forwarding[from] = to;
# }
#
#can_forwaring[0] = 1     rx from bus0 and tx to bu1

import os
import sys

sys.path.append( os.path.abspath("./../panda") )

from time import time, sleep                                                                      
import struct                                                                                     
import argparse                                                                                   
import contextlib                                                                                 
import io                                                                                         
                                                                                                  
from prompt_toolkit import PromptSession                                                          
from prompt_toolkit.history import FileHistory                                                    
from panda import Panda


panda = Panda()

speed_kbps = 500

for bus in range(3):
    panda.set_can_speed_kbps(bus, speed_kbps) 
    panda.set_can_forwarding(bus, -1)   #disable all forwarding
    
#panda.set_safety_mode(Panda.SAFETY_CANGW) 
panda.set_safety_mode(200) 


panda.set_can_forwarding(0,2)
panda.set_can_forwarding(2,0)

panda.close()


opening device 410020000651363038363036 0xddcc
connected


In [35]:
for bus in range(3):
    print(bus)

0
1
2
